In [ ]:
# =================================================================
# --- 0) Initial Setup and Google Drive Integration ---
# =================================================================
# NOTES:
# - Privacy: This runs inside Google Colab. Only YOU can access your Google Drive.
#   Mounting here does not expose your Drive or environment to anyone else.
# - Automation: Input file (transactions) is read directly from Drive, and final
#   Excel with charts/summaries is saved back into the same folder. No manual steps.
# - Input requirements: File should contain at least these columns:
#   ['Date', 'Narration', 'Withdrawal Amt', 'Deposit Amt', 'Closing Balance']
# =================================================================

import pandas as pd
import re
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import numpy as np
from google.colab import drive
from datetime import datetime
import os

# Mount Google Drive to access files
drive.mount('/content/drive', force_remount=True)

# Define the base directory for input/output files
base_drive_path = '/content/drive/MyDrive/Financial_Analysis'
# Create the directory if it doesn't exist
os.makedirs(base_drive_path, exist_ok=True)


# =================================================================
# --- 1) Read the Input File from Google Drive with Validation ---
# =================================================================
# Checks if the given file exists in your Drive folder, validates Excel/CSV formats
# =================================================================

while True:
    input_filename = input("Please enter the name of your transaction file (e.g., Transaction.xls): ")
    input_file_path = os.path.join(base_drive_path, input_filename)

    if not os.path.exists(input_file_path):
        print("\n⛔️ Error: File not found. Please check the file name and try again.")
    else:
        try:
            if input_filename.endswith(('.xls', '.xlsx')):
                df = pd.read_excel(input_file_path)
            else:
                df = pd.read_csv(input_file_path)
            print(f"✅ Successfully read input file from: {input_file_path}")
            break
        except Exception as e:
            print(f"\n⛔️ Error: Could not read the file. Details: {e}")

# =================================================================
# --- 2) Model Loading with Quantization ---
# =================================================================
# NOTE:
# - This part is OPTIONAL and COMMENTED OUT.
# - If you want to fine-tune or run Phi-3-mini locally, uncomment and install.
# - Not needed for core analysis (works fine without it).
# =================================================================

#!pip install --upgrade --force-reinstall transformers accelerate bitsandbytes

#bnb_config = BitsAndBytesConfig(
    #load_in_4bit=True,
    #bnb_4bit_quant_type="nf4",
    #bnb_4bit_compute_dtype=torch.float16,
    #bnb_4bit_use_double_quant=True,
#)

#model_name = "microsoft/Phi-3-mini-4k-instruct"
#tokenizer = AutoTokenizer.from_pretrained(model_name)
#model = AutoModelForCausalLM.from_pretrained(
    #model_name,
    #device_map="auto",
    #quantization_config=bnb_config,
    #torch_dtype=torch.float16,
#)


# =================================================================
# --- 3) Data Pre-processing Functions ---
# =================================================================
# Cleaning & standardizing raw transaction data (amounts, narrations).
# - clean_amount: removes symbols, brackets, text from withdrawal/deposit amounts.
# - normalize_narration: trims narration text for grouping recurring merchants.
# =================================================================

def clean_amount(x):
    if pd.isna(x): return 0.0
    s = str(x).strip()
    if s == "": return 0.0
    s = re.sub(r'\(|\)', '', s)
    s = re.sub(r"[₹$€,]", "", s)
    s = re.sub(r"\bINR\b|\bRs\b|\bCR\b|\bDr\b", "", s, flags=re.IGNORECASE)
    s = re.sub(r"[^0-9\.\-]", "", s)
    if s in ["", "-", ".", "-."]: return 0.0
    try:
        return float(s)
    except:
        return 0.0

def normalize_narration(s):
    if pd.isna(s): return "unknown"
    s = str(s).lower()
    s = re.sub(r'\d+', '', s)
    s = re.sub(r'[^a-z\s]', ' ', s)
    s = re.sub(r'\s+', ' ', s).strip()
    return s[:60]


# =================================================================
# --- 4) Perform Data Analysis and Summarization ---
# =================================================================
# Breakdown:
# - 4A: Monthly & Category Analysis
# - 4B: Compact Transaction Text
# - 4C: Monthly Summaries + % Change
# - 4D: Visualization (Trend Chart - Line Plot)
# - 4E: Visualization (Stacked Bar Chart)
# =================================================================

df['Withdrawal'] = df['Withdrawal Amt'].apply(clean_amount)
df['Deposit'] = df['Deposit Amt'].apply(clean_amount)

total_withdrawals = df['Withdrawal'].sum()
total_deposits = df['Deposit'].sum()

largest_withdrawal, largest_withdrawal_row = 0.0, None
if (df['Withdrawal'] > 0).any():
    idx_w = df['Withdrawal'].idxmax()
    largest_withdrawal = df.loc[idx_w, 'Withdrawal']
    largest_withdrawal_row = df.loc[idx_w, ['Date','Narration','Withdrawal','Closing Balance']]

smallest_withdrawal, smallest_withdrawal_row = 0.0, None
if (df['Withdrawal'] > 0).any():
    idx_s = df[df['Withdrawal'] > 0]['Withdrawal'].idxmin()
    smallest_withdrawal = df.loc[idx_s, 'Withdrawal']
    smallest_withdrawal_row = df.loc[idx_s, ['Date','Narration','Withdrawal','Closing Balance']]

largest_deposit, largest_deposit_row = 0.0, None
if (df['Deposit'] > 0).any():
    idx_d = df['Deposit'].idxmax()
    largest_deposit = df.loc[idx_d, 'Deposit']
    largest_deposit_row = df.loc[idx_d, ['Date','Narration','Deposit','Closing Balance']]

# Recurring patterns
df['norm_narr'] = df['Narration'].apply(normalize_narration)
recurring = (df[df['Withdrawal'] > 0]
              .groupby('norm_narr')
              .agg(count=('Withdrawal','size'), total_spent=('Withdrawal','sum'))
              .sort_values(['count','total_spent'], ascending=False)
              .reset_index()
              .head(10))

# =================================================================
# --- 4A) Monthly & Category Analysis ---
# Handles parsing dates, creating MonthYear, categorizing transactions
# (if 'Category' not in file, auto-assigns based on narration keywords).
# =================================================================
# NOTE: It's recommended to add a 'Category' column manually in your Excel
# for higher accuracy. The auto-categorization is just a fallback.
# =================================================================

# ----------------------------
# Robust Date read + parsing
# ----------------------------

# Read file but keep Date as raw string to avoid early/incorrect Excel parsing
if input_filename.endswith(('.xls', '.xlsx')):
    # use converters so only Date column preserved as string
    df = pd.read_excel(input_file_path, converters={'Date': str})
else:
    # read_csv with Date forced to string
    df = pd.read_csv(input_file_path, dtype={'Date': str})

# Keep a copy of the original date string for debugging/sanity checks
df['_orig_Date_str'] = df['Date'].astype(str)

# Robust parse function: try explicit day-first formats first, then fall back to dayfirst=True
def robust_parse_date(s):
    if pd.isna(s):
        return pd.NaT
    s = str(s).strip()
    if s == "" or s.lower() in ('nan','none','nat'):
        return pd.NaT

    # List of common formats to try (day-first formats first)
    fmts = [
        "%d-%m-%Y", "%d/%m/%Y", "%d.%m.%Y",
        "%Y-%m-%d", "%Y/%m/%d",
        "%m/%d/%Y", "%m-%d-%Y"
    ]
    for fmt in fmts:
        try:
            return pd.to_datetime(s, format=fmt)
        except Exception:
            pass

    # Final fallback: let pandas infer but with dayfirst=True
    try:
        return pd.to_datetime(s, dayfirst=True, errors='coerce')
    except Exception:
        return pd.NaT

# Apply robust parsing and overwrite Date with proper Timestamps
df['Date'] = df['_orig_Date_str'].apply(robust_parse_date)

# Quick sanity print (first 10) so you can see parsed result vs original
print("\nSample original date strings vs parsed Date (first 10 rows):")
print(df[['_orig_Date_str','Date']].head(10).to_string(index=False))

# Now create MonthYear in readable format (e.g., Jul-2025)
df['MonthYear'] = df['Date'].dt.strftime('%b-%Y')

# ----------------------------
# Numeric columns cleaning
# ----------------------------
# (re-use your clean_amount function)
df['Withdrawal'] = df['Withdrawal Amt'].apply(clean_amount)
df['Deposit'] = df['Deposit Amt'].apply(clean_amount)

# ----------------------------
# Category handling (use existing if present)
# ----------------------------
if 'Category' not in df.columns:
    def categorize_transaction(narration):
        narration = str(narration).lower()
        if "loan" in narration: return "Loan"
        elif any(w in narration for w in ["uber","ola","flight","train","bus"]): return "Travel"
        elif any(w in narration for w in ["swiggy","zomato","food","restaurant"]): return "Food"
        elif any(w in narration for w in ["amazon","flipkart","myntra","shopping"]): return "Shopping"
        elif any(w in narration for w in ["netflix","spotify","prime","subscription"]): return "Subscription"
        elif any(w in narration for w in ["payment"]): return "Payment"
        elif any(w in narration for w in ["refund"]): return "Refund"
        else: return "Other"
    df['Category'] = df['Narration'].apply(categorize_transaction)

# ----------------------------
# Monthly category spend (Withdrawal sums)
# ----------------------------
monthly_category_sum = (
    df[df['Withdrawal'] > 0]
      .groupby(['MonthYear','Category'], as_index=False)
      .agg(total_spent=('Withdrawal','sum'))
)

# Ensure MonthYear ordering is sensible — optional: convert MonthYear back to a sortable datetime key
# Create a helper column for true month start (for proper chronological sorting later)
df['MonthStart'] = df['Date'].dt.to_period('M').dt.to_timestamp()
monthly_category_sum = monthly_category_sum.merge(
    df[['MonthYear','MonthStart']].drop_duplicates(subset=['MonthYear']),
    on='MonthYear', how='left'
).sort_values(['MonthStart','Category']).drop(columns=['MonthStart'])

# ----------------------------
# Maximum spent per category (correct alignment)
# ----------------------------
# Sort so the highest withdrawal per category comes first, then pick first() per group
max_spent_per_category = (
    df[df['Withdrawal'] > 0]
      .sort_values(['Category','Withdrawal'], ascending=[True, False])
      .groupby('Category', as_index=False)
      .first()[['Category','Date','Narration','Withdrawal']]
)

# Format Date consistently as dd-mm-YYYY for display/export
max_spent_per_category['Date'] = pd.to_datetime(max_spent_per_category['Date'], errors='coerce').dt.strftime('%d-%m-%Y')

# ----------------------------
# Build transactions_text with formatted date & category
# ----------------------------
def fmt_dt_for_text(ts):
    if pd.isnull(ts):
        return ""
    try:
        return pd.to_datetime(ts).strftime("%d/%m/%Y")
    except:
        return str(ts)

transactions_text = "\n".join([
    f"{fmt_dt_for_text(row['Date'])} | [CATEGORY: {row['Category']}] | {row['Narration'][:60]} | W:{row['Withdrawal']:.2f} | D:{row['Deposit']:.2f} | Bal:{row.get('Closing Balance','')}"
    for _, row in df.head(50).iterrows()
])

# ----------------------------
# Print checks
# ----------------------------
#print("\nMonthly category spend sample:")
#print(monthly_category_sum.head(10).to_string(index=False))
#print("\nMaximum spent per category (sample):")
#print(max_spent_per_category.head(10).to_string(index=False))

# ✅ Fix date alignment for max spent per category
max_spent_per_category = (
    df[df['Withdrawal'] > 0]
      .sort_values(['Category','Withdrawal'], ascending=[True,False])
      .groupby('Category')
      .first()
      .reset_index()[['Category','Date','Narration','Withdrawal']]
)

# ✅ Format Date columns consistently
monthly_category_sum['MonthYear'] = monthly_category_sum['MonthYear']
max_spent_per_category['Date'] = max_spent_per_category['Date'].dt.strftime('%d-%m-%Y')


# =================================================================
# --- 4B) Compact transaction text for LLM ---
# Prepares a short list of transactions as plain text
# (useful if you want to prompt an LLM with structured data).
# =================================================================

transactions_text = "\n".join([
    f"{row['Date']} | {row['Narration'][:60]} | W:{row['Withdrawal']:.2f} | D:{row['Deposit']:.2f} | Bal:{row['Closing Balance']}"
    for _, row in df.head(50).iterrows()
])

numeric_summary = f"""
NUMERIC_SUMMARY:
Total withdrawals = {total_withdrawals:.2f}
Total deposits = {total_deposits:.2f}
Largest withdrawal = {largest_withdrawal:.2f} {('on '+str(largest_withdrawal_row['Date'])+' | '+str(largest_withdrawal_row['Narration']) ) if largest_withdrawal_row is not None else ''}
Smallest withdrawal = {smallest_withdrawal:.2f} {('on '+str(smallest_withdrawal_row['Date'])+' | '+str(smallest_withdrawal_row['Narration']) ) if smallest_withdrawal_row is not None else ''}
Largest deposit = {largest_deposit:.2f} {('on '+str(largest_deposit_row['Date'])+' | '+str(largest_deposit_row['Narration']) ) if largest_deposit_row is not None else ''}
Top recurring merchant patterns:
{recurring.to_string(index=False)}

Monthly spend by category:
{monthly_category_sum.to_string(index=False)}

Maximum spent per category with details:
{max_spent_per_category.to_string(index=False)}
"""

print("✅ Numeric + Monthly summary calculated:\n", numeric_summary)

# =================================================================
# --- 4C) Monthly Summaries + % Change for LLM ---
# Prepares human-readable summaries + calculates % change month-over-month
# =================================================================

# Calculate % change month-to-month per category
monthly_pct_change = (
    monthly_category_sum
    .pivot(index='MonthYear', columns='Category', values='total_spent')
    .pct_change()
    .fillna(0) * 100
)

# Build text summary for LLM
monthly_summary_text = "MONTHLY SUMMARY:\n"
for month, group in monthly_category_sum.groupby('MonthYear'):
    monthly_summary_text += f"\n{month}:\n"
    for _, row in group.iterrows():
        monthly_summary_text += f"  {row['Category']}: {row['total_spent']:.2f}\n"

monthly_summary_text += "\nPERCENTAGE CHANGE (MoM -Increase or Decrease in Spending to Previous Month) :\n"
for month, row in monthly_pct_change.iterrows():
    monthly_summary_text += f"\n{month}:\n"
    for cat, val in row.dropna().items():
        monthly_summary_text += f"  {cat}: {val:.2f}%\n"

print("\n✅ Monthly summary text prepared for LLM:\n")
print(monthly_summary_text)


import io
import matplotlib.pyplot as plt

# =================================================================
# --- 4D) Visualization (Trend Chart - Line Plot) ---
# Category-wise spending over time (withdrawals) plotted as line chart
# =================================================================

print("Category Wise Spending Trend:\n")
fig1, ax1 = plt.subplots(figsize=(10,6))

for cat in monthly_category_sum['Category'].unique():
    cat_data = monthly_category_sum[monthly_category_sum['Category']==cat]
    ax1.plot(cat_data['MonthYear'], cat_data['total_spent'], marker='o', label=cat)

ax1.set_title("Monthly Spending by Category (Withdrawals)")
ax1.set_xlabel("Month-Year")
ax1.set_ylabel("Total Withdrawal")
plt.setp(ax1.get_xticklabels(), rotation=45, ha='right')
ax1.legend(title="Category", bbox_to_anchor=(1.05, 1), loc='upper left')
fig1.tight_layout()


# =================================================================
# --- 4E) Visualization (Stacked Bar Chart) ---
# Category-wise spending stacked for each month (better comparative view)
# =================================================================

print("\nStacked Bar Graph of Category Wise Spending:\n")

pivot_df = (
    monthly_category_sum
    .pivot(index='MonthYear', columns='Category', values='total_spent')
    .fillna(0)
)

# Convert MonthYear to datetime for correct sorting
pivot_df.index = pd.to_datetime(pivot_df.index, format='%b-%Y')
pivot_df = pivot_df.sort_index()

fig2, ax2 = plt.subplots(figsize=(12,8))
pivot_df.plot(kind='bar', stacked=True, ax=ax2)

ax2.set_title("Monthly Spending by Category (Withdrawals)", fontsize=16)
ax2.set_xlabel("Month-Year", fontsize=12)
ax2.set_ylabel("Total Withdrawal", fontsize=12)
plt.setp(ax2.get_xticklabels(), rotation=45, ha='right')
ax2.legend(title="Category", bbox_to_anchor=(1.05, 1), loc='upper left')
fig2.tight_layout()

# =================================================================
# --- 4F) Closing Balance Safety Analysis ---
# =================================================================

print("\nClosing Balance Safety Check:\n")

# Ensure Date is datetime
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

# Get last available closing balance for each month
monthly_closing_balance = (
    df.sort_values('Date')
      .groupby(df['Date'].dt.to_period('M'))
      .last()[['Closing Balance']]
      .reset_index()
)

# Rename columns for clarity
monthly_closing_balance.rename(columns={'Date': 'Month'}, inplace=True)

# Classify safety status
def classify_balance(balance):
    try:
        balance_val = float(str(balance).replace(',', ''))
        return "✅ Safe" if balance_val >= 0 else "⚠️ In Debt"
    except:
        return "Unknown"

monthly_closing_balance['Status'] = monthly_closing_balance['Closing Balance'].apply(classify_balance)

# Print summary
print(monthly_closing_balance.to_string(index=False))


# =================================================================
# --- 5) LLM Prompting and Generation (Optional) ---
# =================================================================
# COMMENTED OUT
# - Uses Phi-3-mini or any other SLM/LLM to generate insights.
# - Relies on summaries prepared in 4B + 4C, so no recalculations inside the LLM.
# =================================================================

#user_prompt = input("\n💬 Enter any additional analysis prompt for the LLM (or press Enter to skip): ")

#messages = [
#   {"role": "system", "content": "You are a financial assistant. Use only the provided summaries, do not invent numbers."},
#   {"role": "user", "content": f"""
#Here are the financial summaries:

#NUMERIC SUMMARY:
#{numeric_summary}

#{monthly_summary_text}

#TRANSACTIONS (sample):
#{transactions_text}

#TASKS:
#1. Compare withdrawals between the last two months and give % change.
#2. Identify which category had the most withdrawal overall.
#3. Give the maximum withdrawal for each category (with date + narration).
#4. Based on category patterns, suggest practical ways to adjust spending.
#5. Keep it concise, actionable, and do not recalc totals.

#IMPORTANT:
#- Only use the numbers provided above.
#- Do not invent values or currency symbols.
#"""}
#]

#def ask_llm(messages, max_new_tokens=400):
#   input_ids = tokenizer.apply_chat_template(
#       messages,
#       add_generation_prompt=True,
#       return_tensors="pt"
#    ).to(model.device)

#    with torch.no_grad():
#        gen_ids = model.generate(
#            input_ids,
#            max_new_tokens=max_new_tokens,
#            do_sample=True,
#            temperature=0.7,
#            top_p=0.9
#       )

#    return tokenizer.decode(gen_ids[0][input_ids.shape[1]:], skip_special_tokens=True)

#analysis = ask_llm(messages, max_new_tokens=400)

#print("\n----- LLM INTERPRETATION -----\n")
#print(analysis)


# =================================================================
# --- 6) Save Outputs + Both Charts to Excel ---
# =================================================================
# - Saves everything into one Excel file in Google Drive.
# - Sheets: NumericSummary, Recurring, MonthlyCategorySum, MaxPerCategory,
#   MonthlySummaryText, MonthlyPctChange, and embedded Charts.
# =================================================================

!pip install xlsxwriter

current_time_str = datetime.now().strftime("%Y%m%d_%H%M%S")
output_filename = f"Spending_Analysis_{current_time_str}.xlsx"
output_file_path = os.path.join(base_drive_path, output_filename)

with pd.ExcelWriter(output_file_path, engine="xlsxwriter") as writer:
    # Numeric summary
    pd.DataFrame([{
        "total_withdrawals": total_withdrawals,
        "total_deposits": total_deposits,
        "largest_withdrawal": largest_withdrawal,
        "smallest_withdrawal": smallest_withdrawal,
        "largest_deposit": largest_deposit
    }]).to_excel(writer, sheet_name="NumericSummary", index=False)

    # Recurring merchants
    recurring.to_excel(writer, sheet_name="Recurring", index=False)

    # Monthly category spend
    monthly_category_sum.to_excel(writer, sheet_name="MonthlyCategorySum", index=False)

    # Max spent per category
    max_spent_per_category.to_excel(writer, sheet_name="MaxPerCategory", index=False)

    # LLM Insights
    #pd.DataFrame([{"LLM_Insights": analysis}]).to_excel(writer, sheet_name="Insights", index=False)


    # Monthly summary text
    pd.DataFrame([{"Monthly_Summary_Text": monthly_summary_text}]).to_excel(
        writer, sheet_name="MonthlySummaryText", index=False
    )

    # Percentage change MoM
    monthly_pct_change.reset_index().to_excel(writer, sheet_name="MonthlyPctChange", index=False)

    # Add to Excel output
    monthly_closing_balance.to_excel(writer, sheet_name="ClosingBalanceCheck", index=False)

    # --- Add charts sheet ---
    workbook = writer.book
    worksheet = workbook.add_worksheet("Charts")
    writer.sheets["Charts"] = worksheet

    # Save fig1 (line chart) into memory and embed
    imgdata1 = io.BytesIO()
    fig1.savefig(imgdata1, format="png", bbox_inches="tight")
    imgdata1.seek(0)
    worksheet.insert_image("B2", "trend_chart.png", {"image_data": imgdata1})

    # Save fig2 (stacked bar) into memory and embed below first chart
    imgdata2 = io.BytesIO()
    fig2.savefig(imgdata2, format="png", bbox_inches="tight")
    imgdata2.seek(0)
    worksheet.insert_image("B25", "stacked_chart.png", {"image_data": imgdata2})

plt.close(fig1)
plt.close(fig2)

print(f"\n✅ Analysis saved to Google Drive as: {output_file_path}")
print("✅ Both charts embedded in 'Charts' sheet")
